In [ ]:
import hashlib
import random
import util
from test_framework.key import ECKey, ECPubKey, jacobi_symbol, SECP256K1_FIELD_SIZE, SECP256K1_ORDER

# 1.1 Introduction to Schnorr Signatures

* Part 1: Schnorr Signatures.
* Part 2: Deterministic Nonces.

## Part 1: Schnorr Signatures

[bip-schnorr](https://github.com/bitcoinops/bips/blob/V0.1/bip-schnorr.mediawiki) defines a signature/verifier scheme, as well as pubkey and signature encodings.

The schnorr signature equation is the following:

* `S = R + H(x(R)|P|msg) * P`

Signing involves generating a secret nonce first.

* Generate secret scalar `k`

Then computing `s` from:

* `s = k - H(x(R)|P|m)`

The resulting signature is:

* `x(R), s`

![test](images/schnorr0.jpg)

### Constraint on the private nonce k

To verify a bip-schnorr signature, the verifier needs the computed `s` scalar and the nonce point `R`. To save 32 bytes in the signature, only the x-value of `R` is provided by the signer, from which the verifier can compute the y-value.

For a given x-coordinate on the secp256k1 curve, there are two possible curve points:

* `y^2 = x^3 + 7` (Two y-coordinate values for a given x-coordinate)
    * For `x`, both `(x, y)` and `(x, -y)` are valid curve points (where `-y` is `SECP256K1_ORDER - y` since all arithmetic is modulo `SECP256K1_ORDER`).
    * One of the y-coordinates is even, and the other is odd (since `SECP256K1_ORDER` is odd).
    * One of the y-coordinates is a quadratic residue (has a square root modulo the field size), and the other is not.

The bip-schnorr proposal constrains `k` such that the y-value of R is a quadratic residue modulo `SECP256K1_ORDER`. This means that from the `x` co-ordinate, the verifier can unambigiously determine `y`.

* `k` and `SECP256K1_ORDER - k` have nonce points `R = (x, y)` and `R = (x, -y)` respectively.
* Only one will have a y-coordinate which is a quadratic residue modulo the field order. If a randomly generated nonce `k` does not yield a valid nonce point `R`, then the signer can negate `k` to obtain a valid nonce.
    
Whether a scalar is a quadratic residue modulo the secp256k1 field order can be determined by its [jacobi symbol](http://en.wikipedia.org/wiki/Jacobi_symbol). The Test Framework provides a `jacobi_symbol()` function. If `jacobi_symbol(y, SECP256K1_FIELD_SIZE) == 1`, then `y` is a quadratic residue.

#### 1.1.1 Example: Calculating a valid nonce

In [ ]:
# Generate a random value in the secp256k1 field
k = random.randrange(1, SECP256K1_ORDER)
print("k = {}\n".format(k))

# Create an uncompressed key so it's easier to access the y-coordinate
k_key = ECKey()
k_key.set(k.to_bytes(32,'big'), False)
R = k_key.get_pubkey()

# Find y and -y
y = int.from_bytes(R.get_bytes()[33:], "big")
minus_y = SECP256K1_FIELD_SIZE - y
print("y = {}".format(y))
print("-y = {}\n".format(minus_y))

# One of y and -y will be even and the other will be odd
print("y is {}".format("odd" if y % 2 else "even"))
print("-y is {}\n".format("odd" if minus_y % 2 else "even"))

# One of y and -y will be a quadratic residue and the other will not
print("y is {}a quadratic residue".format("" if jacobi_symbol(y, SECP256K1_FIELD_SIZE) == 1 else "not "))
print("-y is {}a quadratic residue\n".format("" if jacobi_symbol(minus_y, SECP256K1_FIELD_SIZE) == 1 else "not "))

print("k is {}a valid nonce".format("" if jacobi_symbol(y, SECP256K1_FIELD_SIZE) == 1 else "not "))
print("-k is {}a valid nonce".format("" if jacobi_symbol(minus_y, SECP256K1_FIELD_SIZE) == 1 else "not "))

#### 1.1.2 _Programming Exercise:_ Verify that inverse nonce values `k` and `-k` generate inverse points `R` and `-R`

In [ ]:
# Generate a random value in the secp256k1 field
k = random.randrange(1, SECP256K1_ORDER)
k_key = ECKey()

# Create an uncompressed key from k so it's easier to access the y-coordinate, and get the pubkey


# Find the x- and y-coordinates from R
R_x = # TODO:implement
R_y = # TODO:implement

# Find k's inverse (SECP256K1_FIELD_SIZE - k)

# Create an uncompressed key from -k so it's easier to access the y-coordinate, and get the pubkey


# Find the x- and y-coordinates from -R
minus_R_x = # TODO:implement
minus_R_y = # TODO:implement

assert R_x = minus_R_x
assert SECP256K1_FIELD_SIZE - R_y = minus_R_y

print("Success!")

#### 1.1.3 _Programming Exercise:_ Sign a message with Schnorr

* Sign the message with the provided key pair below.

In [ ]:
msg = hashlib.sha256(b'message').digest()
x = ECKey()
x.generate()
P = x.get_pubkey()

# We have to set uncompressed to get the y-coordinate.
k_int = random.randrange(1, SECP256K1_ORDER)
k = ECKey()
k.set(k_int.to_bytes(32,'big'), False) # Uncompressed.
R = k.get_pubkey()

# Check that nonce is quadratic residue modulo the field order.
# Method: jacobi_symbol(int(y(R)), SECP256K1_FIELD_SIZE)


# Generate s = k + sha256(R_x|P|msg) * x
# Method: hashlib.sha256(bytes)


# Generate sig = R_x|s


# Verify the signature
assert P.verify_schnorr(sig, msg)

print("Success!")

## Part 2: Deterministic Nonces for schnorr signatures

So far we have used a random secret nonce for creating Schnorr signatures. This has the disadvantage that the the user must rely on the robustness of the random generator for each signing rounds. If the nonce generator is biased or even compromised, the private key can be derived for a given signature and known nonce.

For the single signer schnorr signatures, bip-schnorr proposes a deterministic nonce generation scheme.

* `k = sha256(x|msg)`

#### 1.1.4 _Programming Exercise:_ Signing Schnorr with deterministic nonce.

* Create a Schnorr signature with bip-schnorr's deterministic nonce scheme
* Compare this signature to the private key class method `ECKey.sign_schnorr(msg)`

In [ ]:
msg = hashlib.sha256(b'message').digest()
x = ECKey()
x.generate()
P = x.get_pubkey()

# Compute the deterministic nonce.


# Check that nonce is quadratic residue modulo the field order.


# Generate s = k + H(R_x|P|msg) * x


# Generate sig = R_x|s
sig =  # TODO: implement

# Also, generate alternative sig with ECKey.sign_schnorr(msg)
sig2 =  # TODO: implement


# Verify and compare signature(s). 
assert P.verify_schnorr(sig, msg))
assert P.verify_schnorr(sig2, msg)
assert sig == sig2

print("Success!")